Bankruptcy Prediction Analysis

In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

In [2]:
os.getcwd()

'C:\\Users\\adamk\\AppData\\Local\\Programs\\Microsoft VS Code'

In [6]:
url = "https://raw.githubusercontent.com/adam-datasci/RiskAnalytics/main/Data/american_bankruptcy.csv?"
#url = "Data/american_bankruptcy.csv"
df = pd.read_csv(url)

df = df.rename(columns={'X1':'ca','X2':'cogs','X3':'d_and_a','X4':'ebitda','X5':'inventory','X6':'ni','X7':'ar','X8':'mv','X9':'ns','X10':'ta','X11':'tltd','X12':'ebit','X13':'gp','X14':'cl','X15':'re','X16':'tr','X17':'tl','X18':'toe'})

In [7]:
df['wc'] = df['ca'] - df['cl']
df['wc_to_ta'] = df['wc'] / df['ta']
df['re_to_ta'] = df['re'] / df['ta']
df['ebit_to_ta'] = df['ebit'] / df['ta']
df['mv_to_tl'] = df['mv'] / df['tl']
df['tr_to_ta'] = df['tr'] / df['ta']

df['z_score'] = 1.2*df['wc_to_ta'] + 1.4*df['re_to_ta'] + 3.3*df['ebit_to_ta'] + 0.6*df['mv_to_tl'] + 1.0*df['tr_to_ta']

df['latest_period'] = df['year'] == df.groupby('company_name')['year'].transform('max')
df['latest_period'] = df['latest_period'].astype(int)

condition = (df['latest_period'] == 1) & (df['status_label'] == 'failed')
df['default'] = np.where(condition, 1, 0)


In [ ]:
default_periods = df[df['default'] == 1]
non_default_periods = df[df['default'] == 0]
random_sample = non_default_periods.sample(n=609, random_state=1)
df_balanced = pd.concat([default_periods, random_sample])

In [3]:
default_periods.describe().loc[['50%','75%']]
default_periods.describe()


NameError: name 'default_periods' is not defined

In [8]:
non_default_periods.describe().loc[['50%','75%']]

,year,ca,cogs,d_and_a,ebitda,inventory,ni,ar,mv,ns,...,toe,wc,wc_to_ta,re_to_ta,ebit_to_ta,mv_to_tl,tr_to_ta,z_score,latest_period,default
50%,2007.0,101.001,103.705,7.939,15.314,7.051,1.723,22.946,231.6979,186.931,...,169.080,35.016,0.211372,-0.011537,0.048089,2.285556,0.889206,2.755347,0.0,0.0
75%,2012.0,433.678,636.462,48.002,140.634,75.000,40.806,132.389,1259.6438,1051.278,...,878.246,172.174,0.422901,0.275851,0.105426,6.139534,1.455647,5.132444,0.0,0.0


In [9]:
training_features = ['wc_to_ta','re_to_ta','ebit_to_ta','mv_to_tl','tr_to_ta']
training_features = ['wc_to_ta','ebit_to_ta','mv_to_tl','tr_to_ta']

x_train = pd.DataFrame()

for column in training_features:
    x_train[column + '_pct'] = df_balanced[column].rank(method='min', pct=True)

y_train = df_balanced[['default']]

In [10]:
x_train.head(5)

,wc_to_ta_pct,ebit_to_ta_pct,mv_to_tl_pct,tr_to_ta_pct
61,0.270936,0.548440,0.147783,0.467159
192,0.698686,0.473727,0.215107,0.490148
256,0.844007,0.587028,0.640394,0.984401
465,0.264368,0.361248,0.697044,0.067323
595,0.448276,0.703612,0.098522,0.801314


In [11]:
model = sm.Logit(y_train, x_train)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.598391
         Iterations 5


In [12]:
predictions = result.predict(x_train)
predicted_classes = (predictions > 0.5).astype(int)

In [13]:
accuracy = accuracy_score(y_train, predicted_classes)
conf_matrix = confusion_matrix(y_train, predicted_classes)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.6715927750410509
Confusion Matrix:
 [[489 120]
 [280 329]]


In [14]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                 1218
Model:                          Logit   Df Residuals:                     1214
Method:                           MLE   Df Model:                            3
Date:                Sat, 20 Apr 2024   Pseudo R-squ.:                  0.1367
Time:                        18:03:19   Log-Likelihood:                -728.84
converged:                       True   LL-Null:                       -844.25
Covariance Type:            nonrobust   LLR p-value:                 9.166e-50
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
wc_to_ta_pct       0.3034      0.253      1.199      0.231      -0.193       0.800
ebit_to_ta_pct    -1.0851      0.221     -4.915      0.000      -1.518      -0.652
mv_to_tl_pct      -2.0388      0.242     -8.430      0.000      -2.513      -1.565
tr_to_ta_pct       2.1333      0.206     10.381      0.000       1.731       2.536
==================================================================================
"""